In [121]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import h5py
import numpy as np

### Importing custom modules

#### Importing helper

In [122]:
import sys
sys.path.append('./helper')
import hdf5

#### Importing model

In [123]:
sys.path.append('./model')
import faster_r_cnn

### Training model

#### Defining Deep Fashion 2 Dataset

In [124]:
image_dir = "../data/train/image/"
hdf5_path = "../data/train/hdf5/annos.h5"

In [125]:
transform = transforms.Compose([
    transforms.Resize((800, 800)),
    transforms.ToTensor()
])

def convert_index_to_id(index):
    return str(index+1).zfill(6)

def get_image_path(image_dir, index):
    return image_dir + convert_index_to_id(index) + ".jpg"


In [126]:
class DeepFashion2Dataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, hdf5_path,data_length):
        self.image_dir = image_dir
        self.hdf5_path = hdf5_path
        self.data_length = data_length

    def __getitem__(self, index):        
        data_id = convert_index_to_id(index)
        image_path = get_image_path(self.image_dir, index)
        
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image)
        
        with h5py.File(self.hdf5_path, 'r') as h5f:
            bbox = hdf5.extract_attribute(h5f, data_id, 'bounding_box')
            labels = hdf5.extract_attribute(h5f, data_id, 'category_id')
    
        bbox_arr = np.array(bbox,np.float32)
        labels_arr = np.array(labels,np.int64)
    
        target_tensor = {
            "boxes": torch.tensor(bbox_arr, dtype=torch.float32),  
            "labels" : torch.tensor(labels_arr, dtype=torch.int64)
        }

        return image_tensor, target_tensor

    def __len__(self):
        return self.data_length

In [127]:
def collate_fn(batch):
    return tuple(zip(*batch))

with h5py.File(hdf5_path, 'r') as h5f:
    data_length = len(h5f.keys())
    
train_dataset = DeepFashion2Dataset(image_dir, hdf5_path, 2000)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True,collate_fn=collate_fn, num_workers=0)

#### Setting Model

In [128]:
model = faster_r_cnn.FasterRCNNResNet101(num_classes=14)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

FasterRCNNResNet101(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=1e-05)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=1e-05)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=1e-05)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [129]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

#### Looping train

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    
    for i,(images, targets) in enumerate(train_dataloader):        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass and optimization
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

        if i % 100 == 0:        
            print(f"Iteration [{i}/{len(train_dataloader)}], Loss: {losses.item()}")

    # Step the learning rate scheduler
    lr_scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader)}")

print("Training complete!")

#### Saving trained model

In [131]:
torch.save(model.state_dict(), "./model/trained/faster_rcnn_resnet101.pth")